In [1]:
import collections
import re
import d2l.torch as d2l
import torch

### 读取数据集

In [3]:
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt', '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines() # 按行读取
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines] # 将非字母的字符转为空格，再全部小写

lines = read_time_machine()
print(f'# 文本总行数: {len(lines)}')
print(lines[0])
print(lines[10])

# 文本总行数: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


### 词元化

In [4]:
def tokenize(lines, token='word'):
    """将文本行拆分为（单词或字符）词元"""
    assert token == 'word' or token == 'char'
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]

tokens = tokenize(lines) # 词元（token）：文本的基本单位，每个词元都为一个字符串（string）
for i in range(10):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']


### 词表

In [20]:
# 词表vocabulary：将字符串词元映射到从0开始的数字索引中
# 语料corpus：对唯一词元进行统计
# 未知词元<unk> 填充词元<pad> 序列开始词元<bos> 序列结束词元<eos>

def count_cropus(tokens):
    """统计词元的频率"""
    # tokens为一元或二元列表
    if len(tokens) == 0 or isinstance(tokens[0], list):
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

class Vocab:
    """文本词汇本"""
    def __init__(self, tokens=None, min_freq=0, reversed_tokens=None):
        if tokens is None:
            tokens = []
        if reversed_tokens is None:
            reversed_tokens = []

        # 计算每个词元的频数：返回{词元:频数}
        counter = count_cropus(tokens)
        # 将词元列表按频数排序
        self._token_freqs = sorted(counter.items(), key=lambda x:x[1], reverse=True)
        # 加入未知词元标签
        self.idx_to_token = ['<unk>'] + reversed_tokens
        # 将词元标签与数字ID对应，生成dict
        self.token_to_idx = {token: idx for idx, token in enumerate(self.idx_to_token)}
        # 
        
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                # 添加新词元与其索引ID
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1

    @property
    def unk(self):
        return 0
    
    @property
    def token_freqs(self):
        return self._token_freqs
    
    
    def __len__(self):
        """返回词元列表大小"""
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        """返回词元的索引ID"""
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]
        
    def to_tokens(self, indices):
        """返回相应ID对应的词元"""
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]


vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


9

In [6]:
for i in [0, 12]:
    print(f'文本：{tokens[i]}')
    print(f'索引：{vocab.__getitem__(tokens[i])}')
    freq_list = []
    for token in tokens[i]:
        for token1, freq in vocab._token_freqs:
            if token1 == token:
                freq_list.append(freq)
    print(f'频数：{freq_list}')  

文本：['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
索引：[1, 19, 50, 40, 2183, 2184, 400]
频数：[2261, 200, 85, 103, 1, 1, 9]
文本：['lights', 'in', 'the', 'lilies', 'of', 'silver', 'caught', 'the', 'bubbles', 'that', 'flashed', 'and']
索引：[2189, 8, 1, 2190, 4, 589, 363, 1, 2191, 9, 826, 3]
频数：[1, 541, 2261, 1, 1155, 6, 10, 2261, 1, 443, 4, 1245]


## 整合所有功能

In [21]:
def load_corpus_time_machine(max_tokens=-1):
    """返回time machine数据集的词元索引列表和词表"""
    lines = read_time_machine()
    tokens = tokenize(lines, 'char')
    vocab = Vocab(tokens)
    corpus = [vocab[token] for line in tokens for token in line] # vacab[token]返回token对应的ID
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)

(170580, 28)